In [1]:
"""
- aoi
-- faces
-- identification

- input
-- tracking.csv
-- faces.csv
-- metadata.csv

-----
identification_num = 35 / (번호)_(나이)_(성별)_(정서)
faces_num = 49 / (번호)_(나이)_(성별)_(정서)
"""

'\n- aoi\n-- faces\n-- identification\n\n- input\n-- tracking.csv\n-- faces.csv\n-- metadata.csv\n\n-----\nidentification_num = 35 / (번호)_(나이)_(성별)_(정서)\nfaces_num = 49 / (번호)_(나이)_(성별)_(정서)\n'

In [2]:
# 파일 접근 위한 라이브러리
import os
import sys

# 데이터 처리 위한 라이브러리
import pandas as pd
# faces_aoi_converter 에서 나오는 오류 안나오게 하기 위한 코드입니다.
# 기능에는 이상 없으니 걱정 안해도 됩니다.
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

# px2deg, smoothing 위한 라이브러리
import math
from scipy.signal import savgol_filter



In [3]:

"""
결측치 처리를 위한 함수들입니다.

"""


def gap_fill_in(coordinate_data):
    """
    ts(ms) = timestamp
    x(px) = x coordinate
    y(px) = y coordinate
    max_gap_length(ms) = the duration between a last valid point before the gap and a last point in the gap

    """
    
    trial = np.array(coordinate_data['trial'])
    module = np.array(coordinate_data['module'])
    ts = np.array(coordinate_data['timestamp'])
    x = np.array(coordinate_data['x'])
    y = np.array(coordinate_data['y'])
    
    max_gap_length = 300
    
    nan_index = np.argwhere(np.isnan(x))
    continuous_gap = False
    gaps = []
    temp_gap = []

    for i in nan_index:
        
        # continuous_gap이 trial 변화 구간을 포함한 경우, 보정 불가, Gap으로 판단하지 않음
        if continuous_gap == True and trial[i] != trial[i+1]:
            continuous_gap = False
            temp_gap = []
            
        # continuous_gap 정상 종료
        elif continuous_gap == True and (i+1) not in nan_index:
            continuous_gap = False
            temp_gap.append(i)
            gaps.append(temp_gap)
            temp_gap = []
            
        # continuous_gap 정상 지속
        elif continuous_gap == True and (i+1) in nan_index:
            temp_gap.append(i)
            
        # gap의 시작이 첫 index인 경우 보정할 수 없음.. 또한 이후 두개의 elif문에서 i-1를 예외 처리 없게 쓰기위해서 먼저 분리하여 추가
        elif continuous_gap == False and i == 0:
            continue;
            
        # gap의 시작이 각 trial에 첫 index인 경우 보정할 수 없음..
        elif continuous_gap == False and continuous_gap == False and trial[i] != trial[i-1]:
            continue;   
        
       # gap의 시작 전 데이터가 NaN인 경우, 바로 위 Case에서 연속된 경우 역시 보정할 수 없음
        elif continuous_gap == False and continuous_gap == False and (i-1) in nan_index:
            continue;  
            
        # continuous_gap 정상 시작
        elif continuous_gap == False and (i+1) in nan_index and trial[i] == trial[i+1]:
            continuous_gap = True
            temp_gap.append(i)
            
        # gap 1개인 경우, 시작과 동시에 종료
        elif continuous_gap == False and (i+1) not in nan_index and trial[i] == trial[i+1]:
            temp_gap.append(i)
            gaps.append(temp_gap)
            temp_gap = []
            
    for gap in gaps:
        if (ts[np.max(gap)] - ts[np.min(gap)-1]) < max_gap_length:
            cor_data_x =[
                [ts[np.max(gap)+1], x[np.max(gap)+1]],
                [ts[np.min(gap)-1], x[np.min(gap)-1]],
            ]
            cor_data_y = [
                [ts[np.max(gap)+1], y[np.max(gap)+1]],
                [ts[np.min(gap)-1], y[np.min(gap)-1]],
            ]
            for index in gap:
                x[index] = linear_interpolation(cor_data_x, ts[index])
                y[index] = linear_interpolation(cor_data_y, ts[index])
    
    coordinate_data = {
        'timestamp': ts,
        'x': x,
        'y': y,
        'trial': trial,
        'module': module
    }    
    
    return pd.DataFrame.from_dict(coordinate_data)

def linear_interpolation(data, target_x):
    """
    The gap is filled by utilizing linear_interpolation function.
    This is the basic way to interpolate missing data.

    data = list([data_x_1, data_y_1], [data_x_2, data_y_2])
    target_x = x from missing value
    """
    target_y = ((data[0][1] - data[1][1]) \
                / (data[0][0] - data[1][0])) \
                * (target_x - data[1][0]) + data[1][1]
    
    return target_y

In [4]:
"""
Fixation feature 추출을 위한 함수들입니다.

coordinate_data = dict()
coordinate_data[0:3] = list()
"""

def pass_sav_gol_filter(coordinate_data):
    """
    X와 Y 좌표에 sav_gol filter를 적용하는 함수입니다.
    자세한 내용은 다음 document 참고해주시기 바랍니다.
    5와 3은 제가 가장 적절한 값으로 선택해서 넣어놨는데,
    나중에 데이터 만지시면서 더 적합하다고 판단되는 값 있으면 넣으시면 되겠습니다.
    
    Document = https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.savgol_filter.html
    """
    
    coordinate_data['X'] = savgol_filter(coordinate_data['X'], 5, 3)
    coordinate_data['Y'] = savgol_filter(coordinate_data['Y'], 5, 3)

    return coordinate_data

def get_angular_velocity(coordinate_data, px2deg):
    """
    좌표 데이터를 각속도 데이터로 바꾸는 함수입니다.
    각속도 데이터를 np.array 형태로 return 합니다.
    """
    
    x = coordinate_data['X']
    y = coordinate_data['Y']
    ts = coordinate_data['Timestamp']
    velocity = np.array([np.NaN])

    for i in range(len(x)-1):
        degree = px2deg *  math.sqrt(((x[i+1] - x[i]) ** 2) + ((y[i+1] - y[i]) ** 2))

        velocity = np.append(
            velocity,
            
            (degree / (ts[i+1] - ts[i]) * 1000)
        )

    return velocity

def classify_events(velocity_data, velocity_threshold = 30):
    """
    각속도를 입력 받아서 velocity_threshold를 기준으로 움직임을 분류해 인덱스를 리턴합니다.
    """
    velocity_threshold = velocity_threshold
    # For adaptive velocity_threshold
    # velocity_threshold = find_adaptive_threshold(velocity_data, velocity_threshold)

    saccades = np.where(velocity_data > velocity_threshold)[0]
    fixations = np.where(velocity_data <= velocity_threshold)[0]
    blinks = np.where(np.isnan(velocity_data))[0][1:]

    events = {'saccades': saccades,
              'fixations': fixations,
              'blinks': blinks}

# 예시: events = {'saccades': [...], 'fixations': [1, 2, 3, 5, 7, 8], ...}
    for event_key, event in events.items():

        start_index = -1
        end_index = -1
        temp_result = []
        result = []
# 'fixations': [1, 2, 3, 5, 7, 8]
        for i in range(len(event)):
            # 마지막 인덱스에 있는 숫자(8)가 아닐 때 도는 조건문입니다.
            if i != (len(event) - 1):
                # 현재 인덱스와 다음 인덱스의 차이가 1이고, start_index가 디폴트일 때는
                # fixation이 처음 시작하는 인덱스라는 의미이므로, 해당 index를 start_index에 저장합니다.
                # ex. 1, 7
                if ((event[i] + 1) == event[i+1]) and (start_index == -1):
                    start_index = i
                # 현재 인덱스와 다음 인덱스의 차이가 1이고, start_index가 디폴트가 아닐 때는
                # fixation이 이미 시작됐고, 현재 인덱스는 이어지는 인덱스이기 때문에 계속 이어갑니다.
                # ex. 2
                elif ((event[i] + 1) == event[i+1]) and (start_index != -1):
                    continue
                # 현재 인덱스와 다음 인덱스의 차이가 1이 이상이고, start_index가 디폴트가 아닐 때는
                # fixation이 끝난 지점이기 때문에, end_index에 값을 저장하고 result에 인덱스들을 저장합니다.
                # ex. 3
                elif ((event[i] + 1) != event[i+1]) and (start_index != -1):
                    end_index = i+1
                    temp_result = event[start_index: end_index]
                    result.append(temp_result)
                    start_index = -1
                    end_index = -1
                    temp_result = []
                # 현재 인덱스와 다음 인덱스의 차이가 1 이상이며, start_index가 디폴트가 아니므로
                # fixation이 시작과 동시에 끝나는 지점입니다. 바로 result에 넣습니다.
                # ex. 5
                elif ((event[i] + 1) != event[i+1]) and (start_index == -1):
                    result.append([event[i]])
            # 마지막 인덱스에 있는 숫자일 경우 도는 조건문입니다.
            else:
                # start_index가 디폴트라면, 마지막 인덱스이면서 fixation이 시작하는 지점입니다.
                # 시작과 동시에 종료되므로 바로 result에 넣어줍니다.
                # ex. fixation = [1,2,3,5]일 때, 5에 해당함.
                if start_index == -1:
                    result.append([event[i]])
                # 이전의 값과 1 차이가 나고 start_index가 디폴트가 아니기 때문에
                # 이전부터 이어진 fixation index이고, 마지막 인덱스이므로 종료되어야 합니다.
                # 그러므로 현재의 start_index부터 마지막까지 index를 result에 append합니다.
                # ex. 8
                elif (start_index != -1) and ((event[i] - 1) == event[i - 1]):
                    end_index = i
                    result.append(event[start_index:])

        events[event_key] = result

    return events

# For adaptive velocity threshold
def update_threshold(velocity_data, threshold):
    """
    각속도 역치 값을 찾는 함수에서 사용되는 함수입니다.
    velocity_data의 평균 값과 표준편차를 이용해서 최적의 velocity 값을 찾아갑니다.
    다만, velocity_data의 길이가 짧을 경우(시간 span이 짧을 경우) 제대로 작동하지 않습니다.
    따라서, 충분히 데이터가 모인 뒤에 사용해볼 것을 권장합니다.
    """
    velocity_below_threshold = velocity_data[np.where(velocity_data < threshold)]
    mean = velocity_below_threshold.mean()
    std = velocity_below_threshold.std()
    updated_threshold = mean + (std * 6)

    return updated_threshold

def find_adaptive_threshold(velocity_data, threshold):
    """
    update_threshold 함수를 recursive하게 사용해서 최적의 값을 찾습니다.
    threshold 값을 리턴합니다.
    """
    while abs(update_threshold(velocity_data, threshold) - threshold) > 1:
        threshold = update_threshold(velocity_data, threshold)
    return threshold

def get_classification_result(coordinate_data, velocity_data):
    """
    classify_events() 함수에서 return한 인덱스 값을 input으로 받습니다.
    분류된 인덱스를 이용해 시간을 계산합니다.
    label, start_time, end_time, duration, start_x, start_y, end_x, end_y를 리턴합니다.
    """
    result = classify_events(velocity_data)
    timestamp, x, y = coordinate_data['Timestamp'], coordinate_data['X'], coordinate_data['Y']
    output = []
    for key, value in result.items():
        for i in value:
            start_time = timestamp[i[0]-1]
            end_time = timestamp[i[-1]]
            start_x = x[i[0]-1]
            end_x = x[i[-1]]
            start_y = y[i[0]-1]
            end_y = y[i[-1]]
            output.append([key, start_time, end_time, (end_time-start_time), start_x, start_y, end_x, end_y])
    df_output = pd.DataFrame(data = output,
                             columns = ['label', 'start_time', 'end_time', 'duration', 'start_x', 'start_y', 'end_x', 'end_y'])
    df_output.sort_values(by=['start_time'], axis=0, inplace=True)
    return df_output

In [5]:
def load_input(input_file_name):
    temp_data = pd.read_csv(input_dir + '\\{}'.format(input_file_name))

    return temp_data

def get_identification_fixation(trial_result, item, identification_aoi):
    eye_x_aoi_1 = (trial_result["start_x"] >= identification_aoi.at["{}_eye".format(item), 'x1'])
    eye_x_aoi_2 = (trial_result["start_x"] <= identification_aoi.at["{}_eye".format(item), 'x2'])

    eye_x_aoi = (eye_x_aoi_1 & eye_x_aoi_2)

    eye_y_aoi_1 = (trial_result["start_y"] >= identification_aoi.at["{}_eye".format(item), 'y1'])
    eye_y_aoi_2 = (trial_result["start_y"] <= identification_aoi.at["{}_eye".format(item), 'y2'])

    eye_y_aoi = (eye_y_aoi_1 & eye_y_aoi_2)

    mouth_x_aoi_1 = (trial_result["start_x"] >= identification_aoi.at["{}_mouth".format(item), 'x1'])
    mouth_x_aoi_2 = (trial_result["start_x"] <= identification_aoi.at["{}_mouth".format(item), 'x2'])

    mouth_x_aoi = (mouth_x_aoi_1 & mouth_x_aoi_2)

    mouth_y_aoi_1 = (trial_result["start_y"] >= identification_aoi.at["{}_mouth".format(item), 'y1'])
    mouth_y_aoi_2 = (trial_result["start_y"] <= identification_aoi.at["{}_mouth".format(item), 'y2'])

    mouth_y_aoi = (mouth_y_aoi_1 & mouth_y_aoi_2)

    fixation_in_eye_aoi = trial_result[eye_x_aoi & eye_y_aoi]
    fixation_in_mouth_aoi = trial_result[mouth_x_aoi & mouth_y_aoi]

    return fixation_in_eye_aoi, fixation_in_mouth_aoi

def get_faces_fixation(trial_result, faces_aoi):
    face11_x_aoi = ((trial_result["start_x"] > faces_aoi.at['faces_11', 'x1']) & (trial_result["start_x"] < faces_aoi.at['faces_11', 'x2']))
    face11_y_aoi = ((trial_result["start_y"] > faces_aoi.at['faces_11', 'y1']) & (trial_result["start_y"] < faces_aoi.at['faces_11', 'y2']))

    face12_x_aoi = ((trial_result["start_x"] > faces_aoi.at['faces_12', 'x1']) & (trial_result["start_x"] < faces_aoi.at['faces_12', 'x2']))
    face12_y_aoi = ((trial_result["start_y"] > faces_aoi.at['faces_12', 'y1']) & (trial_result["start_y"] < faces_aoi.at['faces_12', 'y2']))

    face21_x_aoi = ((trial_result["start_x"] > faces_aoi.at['faces_21', 'x1']) & (trial_result["start_x"] < faces_aoi.at['faces_21', 'x2']))
    face21_y_aoi = ((trial_result["start_y"] > faces_aoi.at['faces_21', 'y1']) & (trial_result["start_y"] < faces_aoi.at['faces_21', 'y2']))

    face22_x_aoi = ((trial_result["start_x"] > faces_aoi.at['faces_22', 'x1']) & (trial_result["start_x"] < faces_aoi.at['faces_22', 'x2']))
    face22_y_aoi = ((trial_result["start_y"] > faces_aoi.at['faces_22', 'y1']) & (trial_result["start_y"] < faces_aoi.at['faces_22', 'y2']))

    total_fixation = trial_result.label.count()
    face11_fixation = trial_result[face11_x_aoi & face11_y_aoi].label.count()
    face12_fixation = trial_result[face12_x_aoi & face12_y_aoi].label.count()
    face21_fixation = trial_result[face21_x_aoi & face21_y_aoi].label.count()
    face22_fixation = trial_result[face22_x_aoi & face22_y_aoi].label.count()

    return total_fixation, face11_fixation, face12_fixation, face21_fixation, face22_fixation

def extract_identification_dwell_time(tracking_data, item, identification_aoi):
    trial_duration = max(tracking_data['timestamp'])
    all_point_in_trial = tracking_data.shape[0]    

    eye_x_aoi_1 = (tracking_data["x"] >= identification_aoi.at["{}_eye".format(item), 'x1'])
    eye_x_aoi_2 = (tracking_data["x"] <= identification_aoi.at["{}_eye".format(item), 'x2'])

    eye_x_aoi = eye_x_aoi_1 & eye_x_aoi_2

    eye_y_aoi_1 = (tracking_data["y"] >= identification_aoi.at["{}_eye".format(item), 'y1'])
    eye_y_aoi_2 = (tracking_data["y"] <= identification_aoi.at["{}_eye".format(item), 'y2'])

    eye_y_aoi = eye_y_aoi_1 & eye_y_aoi_2

    mouth_x_aoi_1 = (tracking_data["x"] >= identification_aoi.at["{}_mouth".format(item), 'x1'])
    mouth_x_aoi_2 = (tracking_data["x"] <= identification_aoi.at["{}_mouth".format(item), 'x2'])

    mouth_x_aoi = mouth_x_aoi_1 & mouth_x_aoi_2

    mouth_y_aoi_1 = (tracking_data["y"] >= identification_aoi.at["{}_mouth".format(item), 'y1'])
    mouth_y_aoi_2 = (tracking_data["y"] <= identification_aoi.at["{}_mouth".format(item), 'y2'])

    mouth_y_aoi = mouth_y_aoi_1 & mouth_y_aoi_2

    point_in_eye_aoi = tracking_data[eye_x_aoi & eye_y_aoi]
    point_in_mouth_aoi = tracking_data[mouth_x_aoi & mouth_y_aoi]

    abs_dwell_time_eyes = len(point_in_eye_aoi) / all_point_in_trial * trial_duration
    abs_dwell_time_mouth = len(point_in_mouth_aoi) / all_point_in_trial * trial_duration

    try:
        first_time_eye = tracking_data[eye_x_aoi & eye_y_aoi].iat[0,4]
    except:
        first_time_eye = np.nan

    return abs_dwell_time_eyes, abs_dwell_time_mouth, first_time_eye

def extract_faces_dwelltime(trial_tracking_data, faces_aoi):

    trial_duration = max(trial_tracking_data['timestamp'])
    all_point_in_trial = trial_tracking_data.shape[0]   

    face11_x_aoi = (trial_tracking_data["x"] > faces_aoi.at['faces_11', 'x1']) & (trial_tracking_data["x"] < faces_aoi.at['faces_11', 'x2'])
    face11_y_aoi = (trial_tracking_data["y"] > faces_aoi.at['faces_11', 'y1']) & (trial_tracking_data["y"] < faces_aoi.at['faces_11', 'y2'])

    face12_x_aoi = (trial_tracking_data["x"] > faces_aoi.at['faces_12', 'x1']) & (trial_tracking_data["x"] < faces_aoi.at['faces_12', 'x2'])
    face12_y_aoi = (trial_tracking_data["y"] > faces_aoi.at['faces_12', 'y1']) & (trial_tracking_data["y"] < faces_aoi.at['faces_12', 'y2'])

    face21_x_aoi = (trial_tracking_data["x"] > faces_aoi.at['faces_21', 'x1']) & (trial_tracking_data["x"] < faces_aoi.at['faces_21', 'x2'])
    face21_y_aoi = (trial_tracking_data["y"] > faces_aoi.at['faces_21', 'y1']) & (trial_tracking_data["y"] < faces_aoi.at['faces_21', 'y2'])

    face22_x_aoi = (trial_tracking_data["x"] > faces_aoi.at['faces_22', 'x1']) & (trial_tracking_data["x"] < faces_aoi.at['faces_22', 'x2'])
    face22_y_aoi = (trial_tracking_data["y"] > faces_aoi.at['faces_22', 'y1']) & (trial_tracking_data["y"] < faces_aoi.at['faces_22', 'y2'])

    point_in_face11_aoi = trial_tracking_data[face11_x_aoi & face11_y_aoi]
    point_in_face12_aoi = trial_tracking_data[face12_x_aoi & face12_y_aoi]
    point_in_face21_aoi = trial_tracking_data[face21_x_aoi & face21_y_aoi]
    point_in_face22_aoi = trial_tracking_data[face22_x_aoi & face22_y_aoi]

    abs_dwell_time_face11 = len(point_in_face11_aoi) / all_point_in_trial * trial_duration
    abs_dwell_time_face12 = len(point_in_face12_aoi) / all_point_in_trial * trial_duration
    abs_dwell_time_face21 = len(point_in_face21_aoi) / all_point_in_trial * trial_duration
    abs_dwell_time_face22 = len(point_in_face22_aoi) / all_point_in_trial * trial_duration

    return abs_dwell_time_face11, abs_dwell_time_face12, abs_dwell_time_face21, abs_dwell_time_face22

def get_index_of(emotion, faces_trial_to_item):
    result = pd.melt(faces_trial_to_item.loc[:, 'face11':'face22']).value.str.split("_").str[-1] == emotion
    return result

"""
화면 크기에 따라 AOI 영역을 보정해주는 코드입니다.
"""
def identification_aoi_converter(screen_height_resolution, screen_width_resolution, identification_aoi):
    a_screen_height_resolution = 1080
    a_screen_width_resolution = 1920
    b_screen_height_resolution = screen_height_resolution
    b_screen_width_resolution = screen_width_resolution

    resolution_ratio = b_screen_height_resolution / a_screen_height_resolution

    identification_aoi.y1 = identification_aoi.y1 * resolution_ratio
    identification_aoi.y2 = identification_aoi.y2 * resolution_ratio
    identification_aoi.x1 = (b_screen_width_resolution / 2) - (((a_screen_width_resolution / 2) - identification_aoi.x1) * resolution_ratio)
    identification_aoi.x2 = (b_screen_width_resolution / 2) + ((identification_aoi.x2 - (a_screen_width_resolution / 2)) * resolution_ratio)
    
    return identification_aoi

def faces_aoi_converter(screen_height_resolution, screen_width_resolution, faces_aoi):
    
    a_screen_height_resolution = 1080
    a_screen_width_resolution = 1920
    b_screen_height_resolution = screen_height_resolution
    b_screen_width_resolution = screen_width_resolution

    resolution_ratio = b_screen_height_resolution / a_screen_height_resolution

    faces_aoi.y1 = faces_aoi.y1 * resolution_ratio
    faces_aoi.y2 = faces_aoi.y2 * resolution_ratio
    faces_aoi.x1.faces_11 = (b_screen_width_resolution / 2) - (((a_screen_width_resolution / 2) - faces_aoi.x1.faces_11) * resolution_ratio)
    faces_aoi.x2.faces_11 = (b_screen_width_resolution / 2) - (((a_screen_width_resolution / 2) - faces_aoi.x2.faces_11) * resolution_ratio)
    faces_aoi.x1.faces_12 = (b_screen_width_resolution / 2) + ((faces_aoi.x1.faces_12 - (a_screen_width_resolution / 2)) * resolution_ratio)
    faces_aoi.x2.faces_12 = (b_screen_width_resolution / 2) + ((faces_aoi.x2.faces_12 - (a_screen_width_resolution / 2)) * resolution_ratio)
    faces_aoi.x1.faces_21 = (b_screen_width_resolution / 2) - (((a_screen_width_resolution / 2) - faces_aoi.x1.faces_21) * resolution_ratio)
    faces_aoi.x2.faces_21 = (b_screen_width_resolution / 2) - (((a_screen_width_resolution / 2) - faces_aoi.x2.faces_21) * resolution_ratio)
    faces_aoi.x1.faces_22 = (b_screen_width_resolution / 2) + ((faces_aoi.x1.faces_22 - (a_screen_width_resolution / 2)) * resolution_ratio)
    faces_aoi.x2.faces_22 = (b_screen_width_resolution / 2) + ((faces_aoi.x2.faces_22 - (a_screen_width_resolution / 2)) * resolution_ratio)
    
    return faces_aoi

In [6]:
# institute = 'kirbs'
# date = '220623'
# pid = '00000000'

def get_result(institute, date, pid):
    
    # 경로 설정 및 파일 불러오기
    current_dir = os.getcwd()
    institute = institute
    date = date
    pid = pid
    identification_trial_to_item = pd.read_csv(current_dir + "\\{}_{}\\input\\identification\\{}_{}_{}_identification.csv".format(institute, date, institute, date, pid))
    faces_trial_to_item = pd.read_csv(current_dir + "\\{}_{}\\input\\faces\\{}_{}_{}_faces.csv".format(institute, date, institute, date, pid), index_col = 'trial')
    meta_data = pd.read_csv(current_dir + "\\{}_{}\\input\\metadata\\{}_{}_{}_metadata.csv".format(institute, date, institute, date, pid))
    tracking_data = pd.read_csv(current_dir + "\\{}_{}\\input\\tracking\\{}_{}_{}_tracking.csv".format(institute, date, institute, date, pid))
    identification_aoi = pd.read_csv(current_dir + "\\{}_{}\\aoi\\identification.csv".format(institute, date), index_col = 'item')
    faces_aoi = pd.read_csv(current_dir + "\\{}_{}\\aoi\\faces.csv".format(institute, date), index_col = 'item')
    
    # eye tracking 결측치 처리
    # 수정 필요, trial당 gap_fill_in함수로 처리 예정
    # tracking_data = tracking_data.fillna(method='ffill')
    
    tracking_data = gap_fill_in(tracking_data)

    
    # px2deg 계산에 필요한 변수를 선언하겠습니다.
    screen_size = meta_data.screen_size[0]
    distance = meta_data.distance[0]
    screen_height_resolution = meta_data.screen_height_resolution[0]
    screen_width_resolution = meta_data.screen_width_resolution[0]

    # px2deg 계산
    px2deg = math.degrees(math.atan2(0.5 * screen_size, distance)) / (0.5 * screen_height_resolution)
    
    # AOI를 피험자의 해상도에 맞게 변경해주겠습니다.
    identification_aoi = identification_aoi_converter(screen_height_resolution, screen_width_resolution, identification_aoi)
    faces_aoi = faces_aoi_converter(screen_height_resolution, screen_width_resolution, faces_aoi)
    
    # return 파일 선언
    identification_result = pd.DataFrame(columns = [
        'pid',            
        'module',
        'trial',
        'item',
        'total_fixation',
        'eye_fixation',
        'mouth_fixation',
        'abs_dwell_time_eyes',
        'abs_dwell_time_mouth',
        'first_time_eye'])

    faces_result = pd.DataFrame(columns = [
        'pid',
        'module',
        'trial',
        'total_fixation',
        'face11_fixation',
        'face12_fixation',
        'face21_fixation',
        'face22_fixation',
        'total_fixation_under1000',
        'face11_fixation_under1000',
        'face12_fixation_under1000',
        'face21_fixation_under1000',
        'face22_fixation_under1000',
        'total_fixation_over1000',
        'face11_fixation_over1000',
        'face12_fixation_over1000',
        'face21_fixation_over1000',
        'face22_fixation_over1000',
        'abs_dwell_time_face11',
        'abs_dwell_time_face12',
        'abs_dwell_time_face21',
        'abs_dwell_time_face22',
        'abs_dwell_time_face11_under1000',
        'abs_dwell_time_face12_under1000',
        'abs_dwell_time_face21_under1000',
        'abs_dwell_time_face22_under1000',
        'abs_dwell_time_face11_over1000',
        'abs_dwell_time_face12_over1000',
        'abs_dwell_time_face21_over1000',
        'abs_dwell_time_face22_over1000'])
    
    
    # identification 데이터 처리 코드입니다.
    identification_tracking_data = tracking_data[tracking_data.module == "identification"]
    for trial in identification_tracking_data.trial.unique():
        item = identification_trial_to_item.at[trial, 'item']
        identification_trial_tracking_data = identification_tracking_data[identification_tracking_data.trial == trial]
        identification_trial_tracking_data = identification_trial_tracking_data.drop_duplicates(['timestamp', 'module', 'trial'], keep = 'last').reset_index().copy()

        ts = np.array(identification_trial_tracking_data.timestamp)
        x = np.array(identification_trial_tracking_data.x)
        y = np.array(identification_trial_tracking_data.y)

        identification_trial_dict = {'Timestamp': ts,
                                    'X' : x,
                                    'Y' : y}

        identification_trial_dict = pass_sav_gol_filter(identification_trial_dict)
        velocity = get_angular_velocity(identification_trial_dict, px2deg)
        identification_trial_result = get_classification_result(identification_trial_dict, velocity)
        identification_trial_result = identification_trial_result[identification_trial_result.label == 'fixations']

        fixation_in_eye_aoi, fixation_in_mouth_aoi = get_identification_fixation(identification_trial_result, item, identification_aoi)

        abs_dwell_time_eyes, abs_dwell_time_mouth, first_time_eye = extract_identification_dwell_time(identification_trial_tracking_data, item, identification_aoi)

        total_fixation = identification_trial_result.label.count()
        eye_fixation = fixation_in_eye_aoi.label.count()
        mouth_fixation = fixation_in_mouth_aoi.label.count()

        temp_df = pd.DataFrame({
                'pid': pid,
                'module': 'identification',
                'trial': trial,
                'item': item,
                'total_fixation': total_fixation,
                'eye_fixation': eye_fixation,
                'mouth_fixation': mouth_fixation,
                'abs_dwell_time_eyes': abs_dwell_time_eyes,
                'abs_dwell_time_mouth': abs_dwell_time_mouth,
                'first_time_eye': first_time_eye}, index=[0])

        identification_result = pd.concat([identification_result, temp_df])
    identification_result = identification_result.reset_index(drop = True)
 

    
    total_fixation_mean = identification_result['total_fixation'].sum()/len(identification_result)
    eye_fixation_mean = identification_result['eye_fixation'].sum()/len(identification_result)
    mouth_fixation_mean = identification_result['mouth_fixation'].sum()/len(identification_result)
    abs_dwell_time_eyes_mean = identification_result['abs_dwell_time_eyes'].sum()/len(identification_result)
    abs_dwell_time_mouth_mean = identification_result['abs_dwell_time_mouth'].sum()/len(identification_result)

    identification_result_mean = pd.DataFrame({
                                    'pid': pid,
                                    'total_fixation_mean': total_fixation_mean,
                                    'eye_fixation_mean': eye_fixation_mean,
                                    'mouth_fixation_mean': mouth_fixation_mean,
                                    'abs_dwell_time_eyes_mean': abs_dwell_time_eyes_mean,
                                    'abs_dwell_time_mouth_mean': abs_dwell_time_mouth_mean}, index=[0])
                                   
    
    
# faces 데이터 처리 코드입니다.

    faces_tracking_data = tracking_data[tracking_data.module == 'faces']
    for trial in faces_tracking_data.trial.unique():
        # 각 trial의 모든 시간 데이터 분석
        faces_trial_tracking_data = faces_tracking_data[faces_tracking_data.trial == trial]
        faces_trial_tracking_data = faces_trial_tracking_data.drop_duplicates(['timestamp', 'module', 'trial'], keep = 'last').reset_index().copy()

        ts = np.array(faces_trial_tracking_data.timestamp)
        x = np.array(faces_trial_tracking_data.x)
        y = np.array(faces_trial_tracking_data.y)

        faces_trial_dict = {'Timestamp': ts,
                                    'X' : x,
                                    'Y' : y}

        faces_trial_dict = pass_sav_gol_filter(faces_trial_dict)
        velocity = get_angular_velocity(faces_trial_dict, px2deg)
        faces_trial_result = get_classification_result(faces_trial_dict, velocity)
        faces_trial_result = faces_trial_result[faces_trial_result.label == 'fixations']

        # 각 trial의 앞 1초만 데이터 분석

        faces_trial_tracking_data_under1000 = faces_trial_tracking_data[faces_trial_tracking_data.timestamp <= 1000]
        ts_under1000 = np.array(faces_trial_tracking_data_under1000.timestamp)
        x_under1000 = np.array(faces_trial_tracking_data_under1000.x)
        y_under1000 = np.array(faces_trial_tracking_data_under1000.y)

        faces_trial_dict_under1000 = {'Timestamp': ts_under1000,
                                        'X' : x_under1000,
                                        'Y' : y_under1000}

        faces_trial_dict_under1000 = pass_sav_gol_filter(faces_trial_dict_under1000)
        velocity_under1000 = get_angular_velocity(faces_trial_dict_under1000, px2deg)
        faces_trial_result_under1000 = get_classification_result(faces_trial_dict_under1000, velocity_under1000)
        faces_trial_result_under1000 = faces_trial_result_under1000[faces_trial_result_under1000.label == 'fixations']
        
        
        # 각 trial의 1~5초 데이터 분석

        faces_trial_tracking_data_over1000 = faces_trial_tracking_data[faces_trial_tracking_data.timestamp > 1000]
        ts_over1000 = np.array(faces_trial_tracking_data_over1000.timestamp)
        x_over1000 = np.array(faces_trial_tracking_data_over1000.x)
        y_over1000 = np.array(faces_trial_tracking_data_over1000.y)

        faces_trial_dict_over1000 = {'Timestamp': ts_over1000,
                                        'X' : x_over1000,
                                        'Y' : y_over1000}

        faces_trial_dict_over1000 = pass_sav_gol_filter(faces_trial_dict_over1000)
        velocity_over1000 = get_angular_velocity(faces_trial_dict_over1000, px2deg)
        faces_trial_result_over1000 = get_classification_result(faces_trial_dict_over1000, velocity_over1000)
        faces_trial_result_over1000 = faces_trial_result_over1000[faces_trial_result_over1000.label == 'fixations']
       
    
        # 첫 1초 동안 AOI 내에 들어온 Fixation 찾는 코드
        total_fixation_under1000, face11_fixation_under1000, \
        face12_fixation_under1000, face21_fixation_under1000, face22_fixation_under1000 =\
        get_faces_fixation(faces_trial_result_under1000, faces_aoi)

        
        # 1~5초 동안 AOI 내에 들어온 Fixation 찾는 코드
        total_fixation_over1000, face11_fixation_over1000, \
        face12_fixation_over1000, face21_fixation_over1000, face22_fixation_over1000 =\
        get_faces_fixation(faces_trial_result_over1000, faces_aoi)
        
        
        # 자극 시간 전체에 걸쳐 AOI 내에 들어온 Fixation 코드
        total_fixation, face11_fixation, face12_fixation, face21_fixation, face22_fixation =\
        get_faces_fixation(faces_trial_result, faces_aoi)

        
        # 자극 시간 전체에 걸쳐 AOI 내에 들어온 dwelltime 코드
        abs_dwell_time_face11, abs_dwell_time_face12, \
        abs_dwell_time_face21, abs_dwell_time_face22 =\
        extract_faces_dwelltime(faces_trial_tracking_data, faces_aoi)
        
        # 첫 1초 동안 AOI 내에 들어온 dwelltime 코드
        abs_dwell_time_face11_under1000, abs_dwell_time_face12_under1000, \
        abs_dwell_time_face21_under1000, abs_dwell_time_face22_under1000 =\
        extract_faces_dwelltime(faces_trial_tracking_data[faces_trial_tracking_data.timestamp <= 1000], faces_aoi)
        
        # 1~5초 동안 AOI 내에 들어온 dwelltime 코드
        abs_dwell_time_face11_over1000, abs_dwell_time_face12_over1000, \
        abs_dwell_time_face21_over1000, abs_dwell_time_face22_over1000 =\
        extract_faces_dwelltime(faces_trial_tracking_data[faces_trial_tracking_data.timestamp > 1000], faces_aoi)

        temp_df = pd.DataFrame({
                'pid': pid,
                'module': 'faces',
                'trial': trial,
                'total_fixation': total_fixation,
                'face11_fixation': face11_fixation,
                'face12_fixation': face12_fixation,
                'face21_fixation': face21_fixation,
                'face22_fixation': face22_fixation,
                'total_fixation_under1000': total_fixation_under1000,
                'face11_fixation_under1000': face11_fixation_under1000,
                'face12_fixation_under1000': face12_fixation_under1000,
                'face21_fixation_under1000': face21_fixation_under1000,
                'face22_fixation_under1000': face22_fixation_under1000,
                'total_fixation_over1000': total_fixation_over1000,
                'face11_fixation_over1000': face11_fixation_over1000,
                'face12_fixation_over1000': face12_fixation_over1000,
                'face21_fixation_over1000': face21_fixation_over1000,
                'face22_fixation_over1000': face22_fixation_over1000,
                'abs_dwell_time_face11': abs_dwell_time_face11,
                'abs_dwell_time_face12': abs_dwell_time_face12,
                'abs_dwell_time_face21': abs_dwell_time_face21,
                'abs_dwell_time_face22': abs_dwell_time_face22,
                'abs_dwell_time_face11_under1000': abs_dwell_time_face11_under1000,
                'abs_dwell_time_face12_under1000': abs_dwell_time_face12_under1000,
                'abs_dwell_time_face21_under1000': abs_dwell_time_face21_under1000,
                'abs_dwell_time_face22_under1000': abs_dwell_time_face22_under1000,
                'abs_dwell_time_face11_over1000': abs_dwell_time_face11_over1000,
                'abs_dwell_time_face12_over1000': abs_dwell_time_face12_over1000,
                'abs_dwell_time_face21_over1000': abs_dwell_time_face21_over1000,
                'abs_dwell_time_face22_over1000': abs_dwell_time_face22_over1000}, index = [0])

        faces_result = pd.concat([faces_result, temp_df])
    faces_result = faces_result.reset_index(drop = True)
    
    
    
    # Faces 결과를 각 정서마다의 fixation의 합, dwelltime의 합으로 변환하는 코드입니다.
    
    # 정서 종류를 설정합니다.
    emotions = ['neutral', 'happy', 'sad', 'angry']

    # 응시자의 각 trial의 정서를 true, false 값으로 변환합니다.
    # emotion_index는 7개의 key를 가지고 len = 49의 value를 가지는 딕셔너리 값입니다.
    emotion_index = {}

    for emotion in emotions:
        emotion_index[emotion] = get_index_of(emotion,faces_trial_to_item)

    # dict 자료형인 emotion_index를 dataframe으로 만들어줍시다.
    emotion_index_df = pd.concat(emotion_index, axis=1)

    # emotion_index와 faces_result의 값을 곱할 것입니다.
    # faces_result에서 fixation, fixation_under1000, dwelltime 데이터 프레임을 만들어줍시다.
    fixation_series = pd.melt(faces_result.loc[:, 'face11_fixation': 'face22_fixation']).value
    fixation_df = pd.concat([fixation_series]*4, axis=1)
    fixation_df.columns = ['neutral', 'happy', 'sad', 'angry']

    fixation_under1000_series = pd.melt(faces_result.loc[:, 'face11_fixation_under1000': 'face22_fixation_under1000']).value
    fixation_under1000_df = pd.concat([fixation_under1000_series]*4, axis=1)
    fixation_under1000_df.columns = ['neutral', 'happy', 'sad', 'angry']
    
    fixation_over1000_series = pd.melt(faces_result.loc[:, 'face11_fixation_over1000': 'face22_fixation_over1000']).value
    fixation_over1000_df = pd.concat([fixation_over1000_series]*4, axis=1)
    fixation_over1000_df.columns = ['neutral', 'happy', 'sad', 'angry']

    dwelltime_series = pd.melt(faces_result.loc[:, 'abs_dwell_time_face11': 'abs_dwell_time_face22']).value
    dwelltime_df = pd.concat([dwelltime_series]*4, axis=1)
    dwelltime_df.columns = ['neutral', 'happy', 'sad', 'angry']
    
    dwelltime_under1000_series = pd.melt(faces_result.loc[:, 'abs_dwell_time_face11_under1000': 'abs_dwell_time_face22_under1000']).value
    dwelltime_under1000_df = pd.concat([dwelltime_under1000_series]*4, axis=1)
    dwelltime_under1000_df.columns = ['neutral', 'happy', 'sad', 'angry']
    
    dwelltime_over1000_series = pd.melt(faces_result.loc[:, 'abs_dwell_time_face11_over1000': 'abs_dwell_time_face22_over1000']).value
    dwelltime_over1000_df = pd.concat([dwelltime_over1000_series]*4, axis=1)
    dwelltime_over1000_df.columns = ['neutral', 'happy', 'sad', 'angry']

    # emotion_index와 각 feature별 df를 곱해줍시다.
    emotion_fixation_df = emotion_index_df.mul(fixation_df, axis = 0)
    emotion_fixation_under1000_df = emotion_index_df.mul(fixation_under1000_df, axis = 0)
    emotion_fixation_over1000_df = emotion_index_df.mul(fixation_over1000_df, axis = 0)

    emotion_dwelltime_df = emotion_index_df.mul(dwelltime_df, axis = 0)
    emotion_dwelltime_under1000_df = emotion_index_df.mul(dwelltime_under1000_df, axis = 0)
    emotion_dwelltime_over1000_df = emotion_index_df.mul(dwelltime_over1000_df, axis = 0)

    # 이제 각 정서별 feature의 합을 구해봅시다.
    faces_emotion_sum_result = {'fixation' : emotion_fixation_df.sum(),
                         'fixation_under1000' : emotion_fixation_under1000_df.sum(),
                         'fixation_over1000' : emotion_fixation_over1000_df.sum(),
                         'dwelltime' : emotion_dwelltime_df.sum(),
                         'dwelltime_under1000' : emotion_dwelltime_under1000_df.sum(),
                         'dwelltime_over1000' : emotion_dwelltime_over1000_df.sum()}

    # 보기 편하게 행열을 바꿔줍니다.
    faces_emotion_sum_result = pd.DataFrame(faces_emotion_sum_result).transpose()


    # 1*24 데이터프레임으로 변환, 평균값으로 변환
    neutral_fixation_mean = faces_emotion_sum_result.at['fixation', 'neutral']/len(faces_trial_to_item[faces_trial_to_item['q']=='neutral'])
    neutral_fixation_under1000_mean = faces_emotion_sum_result.at['fixation_under1000', 'neutral']/len(faces_trial_to_item[faces_trial_to_item['q']=='neutral'])
    neutral_fixation_over1000_mean = faces_emotion_sum_result.at['fixation_over1000', 'neutral']/len(faces_trial_to_item[faces_trial_to_item['q']=='neutral'])
    neutral_dwelltime_mean = faces_emotion_sum_result.at['dwelltime', 'neutral']/len(faces_trial_to_item[faces_trial_to_item['q']=='neutral'])
    neutral_dwelltime_under1000_mean = faces_emotion_sum_result.at['dwelltime_under1000', 'neutral']/len(faces_trial_to_item[faces_trial_to_item['q']=='neutral'])
    neutral_dwelltime_over1000_mean = faces_emotion_sum_result.at['dwelltime_over1000', 'neutral']/len(faces_trial_to_item[faces_trial_to_item['q']=='neutral'])

    happy_fixation_mean = faces_emotion_sum_result.at['fixation', 'happy']/len(faces_trial_to_item[faces_trial_to_item['q']=='happy'])
    happy_fixation_under1000_mean = faces_emotion_sum_result.at['fixation_under1000', 'happy']/len(faces_trial_to_item[faces_trial_to_item['q']=='happy'])
    happy_fixation_over1000_mean = faces_emotion_sum_result.at['fixation_over1000', 'happy']/len(faces_trial_to_item[faces_trial_to_item['q']=='happy'])
    happy_dwelltime_mean = faces_emotion_sum_result.at['dwelltime', 'happy']/len(faces_trial_to_item[faces_trial_to_item['q']=='happy'])
    happy_dwelltime_under1000_mean = faces_emotion_sum_result.at['dwelltime_under1000', 'happy']/len(faces_trial_to_item[faces_trial_to_item['q']=='happy'])
    happy_dwelltime_over1000_mean = faces_emotion_sum_result.at['dwelltime_over1000', 'happy']/len(faces_trial_to_item[faces_trial_to_item['q']=='happy'])

    sad_fixation_mean = faces_emotion_sum_result.at['fixation', 'sad']/len(faces_trial_to_item[faces_trial_to_item['q']=='sad'])
    sad_fixation_under1000_mean = faces_emotion_sum_result.at['fixation_under1000', 'sad']/len(faces_trial_to_item[faces_trial_to_item['q']=='sad'])
    sad_fixation_over1000_mean = faces_emotion_sum_result.at['fixation_over1000', 'sad']/len(faces_trial_to_item[faces_trial_to_item['q']=='sad'])
    sad_dwelltime_mean = faces_emotion_sum_result.at['dwelltime', 'sad']/len(faces_trial_to_item[faces_trial_to_item['q']=='sad'])
    sad_dwelltime_under1000_mean = faces_emotion_sum_result.at['dwelltime_under1000', 'sad']/len(faces_trial_to_item[faces_trial_to_item['q']=='sad'])
    sad_dwelltime_over1000_mean = faces_emotion_sum_result.at['dwelltime_over1000', 'sad']/len(faces_trial_to_item[faces_trial_to_item['q']=='sad'])

    angry_fixation_mean = faces_emotion_sum_result.at['fixation', 'angry']/len(faces_trial_to_item[faces_trial_to_item['q']=='angry'])
    angry_fixation_under1000_mean = faces_emotion_sum_result.at['fixation_under1000', 'angry']/len(faces_trial_to_item[faces_trial_to_item['q']=='angry'])
    angry_fixation_over1000_mean = faces_emotion_sum_result.at['fixation_over1000', 'angry']/len(faces_trial_to_item[faces_trial_to_item['q']=='angry'])
    angry_dwelltime_mean = faces_emotion_sum_result.at['dwelltime', 'angry']/len(faces_trial_to_item[faces_trial_to_item['q']=='angry'])
    angry_dwelltime_under1000_mean = faces_emotion_sum_result.at['dwelltime_under1000', 'angry']/len(faces_trial_to_item[faces_trial_to_item['q']=='angry'])
    angry_dwelltime_over1000_mean = faces_emotion_sum_result.at['dwelltime_over1000', 'angry']/len(faces_trial_to_item[faces_trial_to_item['q']=='angry'])
    
    faces_emotion_result_reshape = pd.DataFrame({
                'neutral_fixation_mean': neutral_fixation_mean,
                'neutral_fixation_under1000_mean': neutral_fixation_under1000_mean,
                'neutral_fixation_over1000_mean': neutral_fixation_over1000_mean,
                'neutral_dwelltime_mean': neutral_dwelltime_mean,
                'neutral_dwelltime_under1000_mean': neutral_dwelltime_under1000_mean,
                'neutral_dwelltime_over1000_mean': neutral_dwelltime_over1000_mean,
                'happy_fixation_mean': happy_fixation_mean,
                'happy_fixation_under1000_mean': happy_fixation_under1000_mean,
                'happy_fixation_over1000_mean': happy_fixation_over1000_mean,
                'happy_dwelltime_mean': happy_dwelltime_mean,
                'happy_dwelltime_under1000_mean': happy_dwelltime_under1000_mean,
                'happy_dwelltime_over1000_mean': happy_dwelltime_over1000_mean,
                'sad_fixation_mean': sad_fixation_mean,
                'sad_fixation_under1000_mean': sad_fixation_under1000_mean,
                'sad_fixation_over1000_mean': sad_fixation_over1000_mean,
                'sad_dwelltime_mean': sad_dwelltime_mean,
                'sad_dwelltime_under1000_mean': sad_dwelltime_under1000_mean,
                'sad_dwelltime_over1000_mean': sad_dwelltime_over1000_mean,
                'angry_fixation_mean': angry_fixation_mean,
                'angry_fixation_under1000_mean': angry_fixation_under1000_mean,
                'angry_fixation_over1000_mean': angry_fixation_over1000_mean,
                'angry_dwelltime_mean': angry_dwelltime_mean,
                'angry_dwelltime_under1000_mean': angry_dwelltime_under1000_mean,
                'angry_dwelltime_over1000_mean': angry_dwelltime_over1000_mean}, index = [0])
    
    identification_faces_summary = pd.concat([identification_result_mean, faces_emotion_result_reshape], axis = 1)
    
    identification_result.to_csv(current_dir + '\\{}_{}\\output\\identification_result\\{}_{}_{}_identification_result.csv'.format(institute, date, institute, date, pid))
    identification_result_mean.to_csv(current_dir + '\\{}_{}\\output\\identification_result_mean\\{}_{}_{}_identification_result_mean.csv'.format(institute, date, institute, date, pid))
    faces_result.to_csv(current_dir + '\\{}_{}\\output\\faces_result\\{}_{}_{}_faces_result.csv'.format(institute, date, institute, date, pid))
    faces_emotion_result_reshape.to_csv(current_dir + '\\{}_{}\\output\\faces_emotion_sum_result\\{}_{}_{}_faces_emotion_sum_result.csv'.format(institute, date, institute, date, pid))
    identification_faces_summary.to_csv(current_dir + '\\{}_{}\\output\\summary\\{}_{}_{}_identification_faces_summary.csv'.format(institute, date, institute, date, pid))
    
    return identification_faces_summary


In [7]:
def LAI(institute, date):
    
    institute = institute
    date = date
    current_dir = os.getcwd()

    # output을 저장할 폴더를 만들어줍니다.
    os.mkdir(current_dir + "\\{}_{}\\output".format(institute, date))
    os.mkdir(current_dir + "\\{}_{}\\output\\faces_result".format(institute, date))
    os.mkdir(current_dir + "\\{}_{}\\output\\identification_result".format(institute, date))
    os.mkdir(current_dir + "\\{}_{}\\output\\identification_result_mean".format(institute, date))
    os.mkdir(current_dir + "\\{}_{}\\output\\faces_emotion_sum_result".format(institute, date))
    os.mkdir(current_dir + "\\{}_{}\\output\\summary".format(institute, date))

    # input 리스트를 불러옵니다.
    identification_trial_to_item_list = os.listdir(current_dir + "\\{}_{}\\input\\identification".format(institute, date))
    faces_trial_to_item_list = os.listdir(current_dir + "\\{}_{}\\input\\faces".format(institute, date))
    meta_data_list = os.listdir(current_dir + "\\{}_{}\\input\\metadata".format(institute, date))
    tracking_data_list = os.listdir(current_dir + "\\{}_{}\\input\\tracking".format(institute, date))

    # input list를 정렬해줍시다.
    identification_trial_to_item_list.sort()
    faces_trial_to_item_list.sort()
    meta_data_list.sort()
    tracking_data_list.sort()
    
    
    all_summary = pd.DataFrame(columns = [
                    'pid',
                    'total_fixation_mean',
                    'eye_fixation_mean',
                    'mouth_fixation_mean',
                    'abs_dwell_time_eyes_mean',
                    'abs_dwell_time_mouth_mean',
                    'neutral_fixation_mean',
                    'neutral_fixation_under1000_mean',
                    'neutral_fixation_over1000_mean',
                    'neutral_dwelltime_mean',
                    'neutral_dwelltime_under1000_mean',
                    'neutral_dwelltime_over1000_mean',
                    'happy_fixation_mean',
                    'happy_fixation_under1000_mean',
                    'happy_fixation_over1000_mean',
                    'happy_dwelltime_mean',
                    'happy_dwelltime_under1000_mean',
                    'happy_dwelltime_over1000_mean',
                    'sad_fixation_mean',
                    'sad_fixation_under1000_mean',
                    'sad_fixation_over1000_mean',
                    'sad_dwelltime_mean',
                    'sad_dwelltime_under1000_mean',
                    'sad_dwelltime_over1000_mean',
                    'angry_fixation_mean',
                    'angry_fixation_under1000_mean',
                    'angry_fixation_over1000_mean',
                    'angry_dwelltime_mean',
                    'angry_dwelltime_under1000_mean',
                    'angry_dwelltime_over1000_mean'])
    

    # 총 몇 명이 응시했는지 확인해서 변수에 저장합니다.
    overall_participant = len(meta_data_list)

    # 이제 첫 번째 응시자부터 get_result 돌려줍니다.

    for file_index in range(overall_participant):
        pid = meta_data_list[file_index].split('_')[-2]

        identification_faces_summary = get_result(institute, date, pid)
        all_summary = pd.concat([all_summary, identification_faces_summary])
    all_summary.to_csv(current_dir + '\\{}_{}\\output\\summary\\all_summary.csv'.format(institute, date), index=False)
    

In [11]:
LAI('kirbs', '220810')


# 실행할 때마다 회사명, 실시일자를 수정해야 함
# 여기까지 run